In [159]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import skimage.io as io
from skimage import feature
from skimage.transform import resize
from sklearn.feature_selection import VarianceThreshold
from skimage.color import rgb2gray
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from skimage.feature import corner_fast, corner_peaks, corner_harris
import sys
import os
sys.path.append('../')
import helpers.histogram_classifier as hc

In [160]:
def sub_grid(image, size=32):
    """
    Split an image into a grid of non-overlapping square boxes, and return each box in turn
    :params image: image as numpy array
    :params size: side-length of box to use; should be a power of 2
    :return: yields each box in turn
    """
    pic_size = image.shape
    box_side = size
    for x in range(0, pic_size[0], box_side):
        for y in range(0, pic_size[1], box_side):
            yield image[x:x + box_side, y:y + box_side, :]

In [161]:
from IPython.core.display import HTML
css = open('styles/table-style.css').read() #+ open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [162]:
def get_image_features(img, box_size):
    """
    Get RGB and edge features for a single image by splitting the image into a grid of non-overlapping
    square boxes, and obtaining features for each box
    :params img: image as a numpy array
    :params box_size: the side-length in pixels of the box to be used
    :return: mean Red pixel value in each box
    :return: mean Green pixel value in each box
    :return: mean Blue pixel value in each box
    :return: number of edges in each box, extracted using skimage canny edges algorithm
    :return: number of corners in each box, extracted using skimage corner_fast algorithm
    """
    image_features = []
    for box in sub_grid(img, size=box_size):
        features = []
        features.append(np.mean(box[:, :, 0]))
        features.append(np.mean(box[:, :, 1]))
        features.append(np.mean(box[:, :, 2]))
        features.append(np.sum(feature.canny(rgb2gray(box), sigma=1)))
        features.append(corner_peaks(corner_fast(rgb2gray(box))).shape[0])
        image_features.append(np.array(features))
    image_features = np.array(image_features)
    return image_features[:, 0], image_features[:, 1], image_features[:, 2], image_features[:, 3], image_features[:, 4]

In [163]:
def get_complete_features(image_classes, image_names, box_size):
    """
    Get RGB and edge features for a dictionary of image names
    :params image_classes: list of n image classes
    :params image_names: dictionary of m image names for each class
    :params size: size as tuple for reshaping images for extracting edges and corners
    :return: red_features: average red pixel value for each box in each image
    :return: green_features: average green pixel value for each box in each image
    :return: blue_features: average blue pixel value for each box in each image
    :return: edge_features: number of edges detected for each box in each image
    :return: corner_features: number of corners detected for each box in each image
    :return: labels: array of true labels for each image
    """
    red_features = []
    green_features = []
    blue_features = []
    edge_features = []
    corner_features = []
    labels = []
    for c in image_classes:
        for i in image_names[c]:
            img = io.imread(os.path.join("../data/food-101/top_classes/", c, "", i))
            red, green, blue, edge, corners = get_image_features(img, box_size=box_size)
            red_features.append(red)
            green_features.append(green)
            blue_features.append(blue)
            edge_features.append(edge)
            corner_features.append(corners)
            labels.append(c)
    
    return np.array(red_features), np.array(green_features), np.array(blue_features), np.array(edge_features), np.array(corner_features), np.array(labels)

In [164]:
BOX = 32

In [165]:
classes = pd.read_csv('../data/food-101/meta/top_classes.csv', index_col=0)
class_list = list(classes['class'].unique())

image_names = {}
for c in class_list:
    image_names[c] = np.array(classes[classes['class'] == c]['name'])

In [166]:
train, test = hc.split_data(image_names, 0.75)

In [167]:
#for c in class_list:
#    train[c] = train[c][:15]
#    test[c] = test[c][:5]

In [168]:
#Get complete histogram features, edge and corners
all_red_features, all_green_features, all_blue_features, all_edge_features, all_corner_features, train_labels = get_complete_features(class_list, train, BOX)

#### Classifier for red features

In [169]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(all_red_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 8, 10, 12, 14]), 'n_estimators': array([ 10,  20,  50, 100, 200])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [170]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.2078888888888889
Params: {'max_depth': 12, 'n_estimators': 200}


In [171]:
%%time
red_rf = grid.best_estimator_
red_rf.fit(all_red_features, train_labels)

CPU times: user 21.8 s, sys: 51.3 ms, total: 21.8 s
Wall time: 21.9 s


#### Classifier for green features

In [172]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(all_green_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 8, 10, 12, 14]), 'n_estimators': array([ 10,  20,  50, 100, 200])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [173]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.225
Params: {'max_depth': 14, 'n_estimators': 200}


In [174]:
%%time
green_rf = grid.best_estimator_
green_rf.fit(all_green_features, train_labels)

CPU times: user 23.3 s, sys: 89.9 ms, total: 23.4 s
Wall time: 23.4 s


#### Classifier for blue features

In [175]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(all_blue_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 8, 10, 12, 14]), 'n_estimators': array([ 10,  20,  50, 100, 200])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [176]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.19922222222222222
Params: {'max_depth': 10, 'n_estimators': 200}


In [177]:
%%time
blue_rf = grid.best_estimator_
blue_rf.fit(all_blue_features, train_labels)

CPU times: user 18.5 s, sys: 24.5 ms, total: 18.6 s
Wall time: 18.6 s


#### Classifier for edge features

In [178]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(all_edge_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 8, 10, 12, 14]), 'n_estimators': array([ 10,  20,  50, 100, 200])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [179]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.21922222222222223
Params: {'max_depth': 12, 'n_estimators': 200}


In [180]:
%%time
edge_rf = grid.best_estimator_
edge_rf.fit(all_edge_features, train_labels)

CPU times: user 13.5 s, sys: 74.1 ms, total: 13.5 s
Wall time: 13.6 s


#### Classifier for corner features

In [181]:
#Setup grid-search for top parameters
depth = np.array([8, 10, 12, 14])
estimators = np.array([10, 20, 50, 100, 200])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(n_estimators=estimators, max_depth=depth))
grid.fit(all_corner_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([ 8, 10, 12, 14]), 'n_estimators': array([ 10,  20,  50, 100, 200])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [182]:
print("Best score = {}".format(grid.best_score_))
print("Params: {}".format(grid.best_params_))

Best score = 0.17833333333333334
Params: {'max_depth': 14, 'n_estimators': 200}


In [183]:
%%time
corner_rf = grid.best_estimator_
corner_rf.fit(all_corner_features, train_labels)

CPU times: user 4.75 s, sys: 46.4 ms, total: 4.79 s
Wall time: 4.8 s


In [184]:
red_features = red_rf.predict_proba(all_red_features)
green_features = green_rf.predict_proba(all_green_features)
blue_features = blue_rf.predict_proba(all_blue_features)
edge_features = edge_rf.predict_proba(all_edge_features)
corner_features = corner_rf.predict_proba(all_corner_features)

combined_features = np.concatenate([red_features, green_features, blue_features, edge_features, corner_features], axis=1)

##### Get test features

In [185]:
all_red_features_t, all_green_features_t, all_blue_features_t, all_edge_features_t, all_corner_features_t, test_labels = get_complete_features(class_list, test, BOX)

In [186]:
red_predictions = red_rf.predict_proba(all_red_features_t)
green_predictions = green_rf.predict_proba(all_green_features_t)
blue_predictions = blue_rf.predict_proba(all_blue_features_t)
edge_predictions = edge_rf.predict_proba(all_edge_features_t)
corner_predictions = corner_rf.predict_proba(all_corner_features_t)

combined_predictions = np.concatenate([red_predictions, green_predictions, blue_predictions, edge_predictions, corner_predictions], axis=1)

#### Bayesian Net for combining predictions

In [187]:
nb = GaussianNB()
nb.fit(combined_features, train_labels)

GaussianNB()

In [188]:
%%time
predicted_labels = nb.predict(combined_predictions)

CPU times: user 12.4 ms, sys: 4.87 ms, total: 17.2 ms
Wall time: 16.1 ms


In [189]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted_labels, class_list)

In [190]:
overall

,Accuracy,F1,Precision,Recall
Results,0.287,0.286,0.294,0.287


In [191]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.30,0.30,0.30,250.0
lasagna,0.22,0.23,0.23,250.0
french_toast,0.30,0.23,0.26,250.0
guacamole,0.28,0.25,0.26,250.0
apple_pie,0.19,0.27,0.22,250.0
cheesecake,0.39,0.36,0.37,250.0
hamburger,0.25,0.16,0.20,250.0
fried_rice,0.37,0.50,0.43,250.0
carrot_cake,0.22,0.30,0.25,250.0
chocolate_cake,0.32,0.34,0.33,250.0


In [192]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,76,24,20,10,6,4,6,7,20,28,43,6
lasagna,19,58,17,15,31,10,7,28,22,7,20,16
french_toast,19,32,58,7,29,10,9,20,24,12,18,12
guacamole,14,10,11,63,30,12,25,37,17,9,6,16
apple_pie,6,23,9,18,68,17,16,25,51,8,1,8
cheesecake,4,9,3,6,54,89,8,2,34,32,6,3
hamburger,7,24,16,32,41,10,40,20,29,21,8,2
fried_rice,9,19,5,18,29,1,12,125,9,3,2,18
carrot_cake,22,18,11,12,33,20,9,13,75,23,9,5
chocolate_cake,19,4,17,4,10,35,10,1,33,85,29,3


##### SVM for combining predictions

In [193]:
#Setup grid-search for top parameters
param_grid = [
  {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'kernel': ['linear']},
  {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']},
]
model = SVC()
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid.fit(combined_features, train_labels)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}, {'kernel': ['rbf'], 'C': [0.001, 0.01, 0.1, 1, 10, 100], 'gamma': [1, 0.01, 0.001, 0.0001]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [194]:
print(grid.best_score_)
print(grid.best_params_)

0.999777777778
{'kernel': 'linear', 'C': 100}


In [195]:
%%time
sv = grid.best_estimator_
sv.fit(combined_features, train_labels)

CPU times: user 91.8 ms, sys: 717 µs, total: 92.5 ms
Wall time: 91.7 ms


In [196]:
%%time
predicted_labels = sv.predict(combined_predictions)

CPU times: user 36.8 ms, sys: 812 µs, total: 37.6 ms
Wall time: 36.4 ms


In [197]:
overall, metrics, confusion = hc.get_metrics(test_labels, predicted_labels, class_list)

In [198]:
overall

,Accuracy,F1,Precision,Recall
Results,0.284,0.272,0.3,0.284


In [199]:
metrics

,Precision,Recall,F1-Score,Support
pork_chop,0.28,0.27,0.28,250.0
lasagna,0.32,0.11,0.16,250.0
french_toast,0.24,0.32,0.27,250.0
guacamole,0.31,0.26,0.28,250.0
apple_pie,0.31,0.07,0.12,250.0
cheesecake,0.33,0.41,0.37,250.0
hamburger,0.26,0.08,0.12,250.0
fried_rice,0.49,0.44,0.46,250.0
carrot_cake,0.18,0.38,0.24,250.0
chocolate_cake,0.23,0.38,0.28,250.0


In [200]:
confusion

,pork_chop,lasagna,french_toast,guacamole,apple_pie,cheesecake,hamburger,fried_rice,carrot_cake,chocolate_cake,steak,pizza
pork_chop,68,8,21,3,2,2,2,6,41,39,46,12
lasagna,16,27,33,18,9,20,4,18,33,20,26,26
french_toast,22,7,81,5,4,19,5,8,38,17,21,23
guacamole,4,5,18,64,1,19,5,24,40,37,13,20
apple_pie,9,6,36,23,18,39,10,15,58,8,7,21
cheesecake,10,6,11,12,4,103,6,7,41,38,10,2
hamburger,9,12,41,25,6,13,20,16,55,36,10,7
fried_rice,7,4,16,23,3,11,5,109,23,17,3,29
carrot_cake,20,2,28,10,3,32,6,6,94,32,9,8
chocolate_cake,25,2,12,1,0,31,6,1,47,94,30,1
